In [1]:
import pandas as pd
import spacy
import gensim
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import regex as re
import nltk
from nltk.corpus import stopwords
import string
from collections import defaultdict
import numpy as np

Reproducing th elexical analysis done in Aguirre ert al using log odd rario informative direlicht prior
- use LIWC tcategroy count instead of using vocab count
- use log of frequency

In [2]:
df1 = pd.read_csv('/home/shinkamo/598sdl/data/synthetic/1.csv')
df2 = pd.read_csv('/home/shinkamo/598sdl/data/synthetic/2.csv')
df3 = pd.read_csv('/home/shinkamo/598sdl/data/synthetic/3.csv')
df = pd.concat([df1, df2, df3])
df.head()

,Unnamed: 0,[name],[race],[psych],[gender],[text]
0,0,Chloe,a white,PTSD,f,I am tired of the lack of diversity in the te...
1,1,Chloe,a white,depression,f,"I have a lot of privilege. I'm well-educated,..."
2,2,Chloe,a black,PTSD,f,I am constantly feeling like I'm not good eno...
3,3,Chloe,a black,depression,f,I often feel like I have to be twice as good ...
4,4,Chloe,an asian,PTSD,f,I have felt the immense pressure to succeed i...


In [3]:
wordnet_lemmatizer = WordNetLemmatizer()
englishStemmer=SnowballStemmer("english")

def clean_text(x):
    temp_text = x.lower()
    # Remove punctuation of the string
    temp_text = temp_text.translate(str.maketrans('', '', string.punctuation))
    # Remove new line from string
    temp_text = temp_text.replace('\n', '')
    # Remove double space or more
    temp_text = re.sub(' +', ' ', temp_text).strip()
    # Tokenized the text
    temp_text = nltk.word_tokenize(temp_text)
    stop_words = set(stopwords.words('english'))
    
    filtered_word = []
    
    for word in temp_text:
        # Lemmanize and stem word
        lemma_word = wordnet_lemmatizer.lemmatize(word)
        stemmed_word = englishStemmer.stem(lemma_word)
        
        # Do not add stop words into the the final cleaned sentence
        if stemmed_word in stop_words:
            continue
        else:
            filtered_word.append(stemmed_word)
            
    return " ".join(filtered_word).strip()

In [4]:
def load_liwc():
    """ Load LIWC data """

    with open("/home/public/LinguisticEthnography/LIWC/LIWC.2015.all.txt", "r") as file_p:
        data = file_p.readlines()
    liwc = defaultdict(list)
    for entry in data:
        entry = entry.split(' ,')
        liwc[entry[1].rstrip('\n')].append(entry[0])
    return liwc

class LIWC:
    def __init__(self):
        self.liwc = load_liwc()

    def count_liwc(self, sentence, categories=None):
        """
        Count liwc categories
        """
        if not categories:
            categories = []

        sentence = clean_text(sentence)
        counts = {}
        for category, tokens in self.liwc.items():
            counts[category] = sum([sentence.count(_tok) for _tok in tokens])
        return counts

Getting LI count per category(gender)

In [5]:
fem = df[df['[gender]'] == 'f']
fem_text = ""
for t in fem['[text]']:
    fem_text += t


In [6]:
liwc = LIWC()
f_liwc = liwc.count_liwc(fem_text)

In [7]:
masc = df[df['[gender]'] == 'm']
masc_text = ""
for t in masc['[text]']:
    masc_text += t
m_liwc = liwc.count_liwc(masc_text)

# Parameters
- y - wc for topic
- a = prior, estimated by avg count pe category

groups = female, male, i
topic = liwc, k

In [8]:
liwc = LIWC()

In [9]:
liwc.liwc['I']

['i',
 "i'd",
 "i'd've",
 "i'll",
 "i'm",
 "i've",
 'id',
 'idc',
 'idgaf',
 'idk',
 'idontknow',
 'idve',
 'ikr',
 'ily*',
 'im',
 'ima',
 'imean',
 'imma',
 'ive',
 'me',
 'methinks',
 'mine',
 'my',
 'myself']

In [10]:
health = ['MONEY', 'REWARD']
relationships = ['FAMILY', 'HOME']
selves = ['I']
social = ['SHEHE', 'YOU', 'WE', 'THEY']
  

In [11]:
alpha = np.mean(list(f_liwc.values()))
n  = np.sum(list(f_liwc.values())) + np.sum(list(m_liwc.values()))

In [12]:
f_dict = defaultdict(list,{ k:[] for k in liwc.liwc.keys() })

In [13]:
cat = health
n_ik = np.sum([f_liwc[i] for i in cat])
n_k = np.sum([m_liwc[i] for i in cat]) + np.sum([m_liwc[i] for i in cat])


for c in cat:
    y_ikw = f_liwc[c]
    y_kw = f_liwc[c] + m_liwc[c]
    a_ikw = (alpha/n)*y_ikw
    a_kw = (alpha/n)*y_kw

    log_odd = np.log((y_ikw + a_ikw)/(n + alpha- y_kw - a_ikw)) - np.log((y_kw  + a_kw)/(n_k+ alpha - y_kw- a_kw))
    f_dict[c] = log_odd



In [19]:
def calculate_log_odd(cat, f_dict, i_liwc, j_liwc):
    n_ik = np.sum([i_liwc[i] for i in cat])
    n_k = np.sum([j_liwc[i] for i in cat]) + np.sum([j_liwc[i] for i in cat])


    for c in cat:
        y_ikw = i_liwc[c]
        y_kw = i_liwc[c] + j_liwc[c]
        a_ikw = (alpha/n)*y_ikw
        a_kw = (alpha/n)*y_kw

        log_odd = np.log((y_ikw + a_ikw)/(n_ik + alpha- y_kw - a_ikw)) - np.log((y_kw  + a_kw)/(n_k+ alpha - y_kw- a_kw))
        f_dict[c] = log_odd

    return f_dict

In [26]:
f_dict = defaultdict(list,{ k:[] for k in liwc.liwc.keys() })

lo_health = calculate_log_odd(health, f_dict, f_liwc, m_liwc)
lo_rel = calculate_log_odd(relationships, lo_health,f_liwc,m_liwc)
lo_selves = calculate_log_odd(relationships, lo_rel,f_liwc,  m_liwc)
f_lo = calculate_log_odd(relationships, lo_selves,f_liwc, m_liwc)



In [27]:
lo_social


defaultdict(list,
            {'RELIG': [],
             'BODY': [],
             'COMPARE': [],
             'MONEY': -0.34171488931504923,
             'PRONOUN': [],
             'FEEL': [],
             'CERTAIN': [],
             'INSIGHT': [],
             'ASSENT': [],
             'NUMBER': [],
             'NEGATE': [],
             'SAD': [],
             'AFFILIATION': [],
             'COGPROC': [],
             'FEMALE': [],
             'AFFECT': [],
             'ANGER': [],
             'HOME': -0.3952751313103329,
             'CONJ': [],
             'SEXUAL': [],
             'NEGEMO': [],
             'PPRON': [],
             'DIFFER': [],
             'DEATH': [],
             'FAMILY': -0.5015726742987066,
             'ADVERB': [],
             'SPACE': [],
             'INFORMAL': [],
             'POSEMO': [],
             'ANX': [],
             'FOCUSPRESENT': [],
             'NETSPEAK': [],
             'PERCEPT': [],
             'HEALTH': [],
           

In [28]:
m_dict = defaultdict(list,{ k:[] for k in liwc.liwc.keys() })

lo_health = calculate_log_odd(health, m_dict,  m_liwc, f_liwc)
lo_rel = calculate_log_odd(relationships, lo_health,m_liwc,f_liwc)
lo_selves = calculate_log_odd(relationships, lo_rel,m_liwc, f_liwc)
m_lo = calculate_log_odd(relationships, lo_selves,m_liwc, f_liwc)


In [29]:
lo_social

defaultdict(list,
            {'RELIG': [],
             'BODY': [],
             'COMPARE': [],
             'MONEY': -0.34171488931504923,
             'PRONOUN': [],
             'FEEL': [],
             'CERTAIN': [],
             'INSIGHT': [],
             'ASSENT': [],
             'NUMBER': [],
             'NEGATE': [],
             'SAD': [],
             'AFFILIATION': [],
             'COGPROC': [],
             'FEMALE': [],
             'AFFECT': [],
             'ANGER': [],
             'HOME': -0.3952751313103329,
             'CONJ': [],
             'SEXUAL': [],
             'NEGEMO': [],
             'PPRON': [],
             'DIFFER': [],
             'DEATH': [],
             'FAMILY': -0.5015726742987066,
             'ADVERB': [],
             'SPACE': [],
             'INFORMAL': [],
             'POSEMO': [],
             'ANX': [],
             'FOCUSPRESENT': [],
             'NETSPEAK': [],
             'PERCEPT': [],
             'HEALTH': [],
           